In [175]:
import pandas as pd
from scipy.stats import randint, uniform

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import pickle

In [176]:
project_accidents_df = pd.read_csv('../data/clean_df.csv')
project_accidents_df = project_accidents_df.sample(20000, random_state=42)

In [177]:
pd.set_option('display.max_columns', None)
project_accidents_df.head(2)

,date,day_of_week,hour,accident_number,road_category,road_layout,num_lanes,reserved_lane,road_profile,road_shape,surface_condition,infrastructure,road_location,speed_limit,injury_severity,users_involved,light_conditions,department,urban_area,intersection_type,weather,collision_type,latitude,longitude,region
26316,2019-01-23,Wednesday,21,201900028031,Local & Access Roads,One Way,4,No value,Flat,Straight,Normal,No value,Road,70,1,3,Night,75,Inside urban area,No junction,Normal,Multi-car collision,48.88343,2.399739,Île-de-France
148184,2021-01-28,Thursday,16,202100051164,Secondary Roads,Two Way,2,No value,Flat,Straight,Normal,No value,Road,50,2,3,Day,63,Inside urban area,No junction,Normal,Multi-car collision,45.87138,3.133130,Auvergne-Rhône-Alpes


In [178]:
X = project_accidents_df.drop(columns=['injury_severity','date', 'accident_number','department', 'latitude', 'longitude', 'region'])
y = project_accidents_df['injury_severity']


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
X.shape, y.shape

((20000, 18), (20000,))

In [180]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [181]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16000, 18), (4000, 18), (16000,), (4000,))

In [183]:
from sklearn.compose import make_column_selector

num_col = make_column_selector(dtype_include=['int64', 'float64'])
cat_col = make_column_selector(dtype_include=['object'])


num_transformer = Pipeline([
    ('standard_scaler', StandardScaler())
])

cat_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor= ColumnTransformer([
    ('num_transformer', num_transformer, num_col),
    ('cat_transformer', cat_transformer, cat_col)
])

In [184]:
preprocessor

,transformers,"[('num_transformer', ...), ('cat_transformer', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


---

## Modeling

###  Default Logistic Regression

In [186]:
# Logistic Regression

model_lr = LogisticRegression(max_iter=1000)

pipeline_lr = make_pipeline(preprocessor, model_lr)
cv_results = cross_val_score(pipeline_lr, X_test, y_test, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7317500000000001)

### Optimize Logistic Regression

In [187]:
pipeline_lr.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num_transformer',
                                    Pipeline(steps=[('standard_scaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x1317824b0>),
                                   ('cat_transformer',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x131782210>)])),
  ('logisticregression', LogisticRegression(max_iter=1000))],
 'transform_input': None,
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('num_transformer',
                                  Pipeline(steps=[('standard_scaler',
                                                   StandardScaler())]),
                                  <sklearn.compose._co

In [190]:
params = {
    'logisticregression__C': uniform(0.0001, 10),
    #'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__solver': ['lbfgs', 'saga'],
}

search_lr = RandomizedSearchCV(
    pipeline_lr,
    params,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=42)

search_lr.fit(X, y)
search_lr.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/yaeyukimoto/.pyenv/versions/3.12.9/envs/project_accidents/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/yaeyukimoto/.pyenv/versions/3.12.9/envs/project_accidents/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/yaeyukimoto/.pyenv/versions/3.12.9/envs/project_accidents/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/yaeyukimoto/.pyenv/versions/3.12.9/envs/project_accidents/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'logisticregression__C': np.float64(0.007887658410143283),
 'logisticregression__solver': 'saga'}

In [191]:
best_pipeline_lr = search_lr.best_estimator_

cv_results = cross_val_score(best_pipeline_lr, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7404375000000001)

### Default KNN

In [193]:
# KNN
model_knn = KNeighborsClassifier(n_neighbors=5)

pipeline_knn = make_pipeline(preprocessor, model_knn)
cv_results = cross_val_score(pipeline_knn, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7116875)

### Optimize KNN

In [194]:
pipeline_knn.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num_transformer',
                                    Pipeline(steps=[('standard_scaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x1317824b0>),
                                   ('cat_transformer',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x131782210>)])),
  ('kneighborsclassifier', KNeighborsClassifier())],
 'transform_input': None,
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('num_transformer',
                                  Pipeline(steps=[('standard_scaler',
                                                   StandardScaler())]),
                                  <sklearn.compose._column_tran

In [195]:
params = {
    "kneighborsclassifier__n_neighbors": randint(1,50),
}

search_knn = RandomizedSearchCV(
                        pipeline_knn,
                        params,
                        n_iter=50,
                        n_jobs=-1,
                        cv=5,
                        scoring='accuracy',
                        verbose=1,
                        random_state=42)

search_knn.fit(X_train,y_train)
search_knn.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


{'kneighborsclassifier__n_neighbors': 39}

In [196]:
best_pipeline_knn = search_knn.best_estimator_

cv_results = cross_val_score(best_pipeline_knn, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7414375)

### Default Decision Tree

In [197]:
# Decision Tree
model_tree = DecisionTreeClassifier(max_depth=2, random_state=42)

pipeline_tree = make_pipeline(preprocessor, model_tree)
cv_results = cross_val_score(pipeline_tree, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.729875)

### Optimize Decision Tree

In [198]:
pipeline_tree.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num_transformer',
                                    Pipeline(steps=[('standard_scaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x1317824b0>),
                                   ('cat_transformer',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x131782210>)])),
  ('decisiontreeclassifier',
   DecisionTreeClassifier(max_depth=2, random_state=42))],
 'transform_input': None,
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('num_transformer',
                                  Pipeline(steps=[('standard_scaler',
                                                   StandardScaler())]),
                            

In [199]:
params = {
    'decisiontreeclassifier__max_depth': randint(2, 20),
    'decisiontreeclassifier__min_samples_split': randint(2, 50),
    'decisiontreeclassifier__min_samples_leaf': randint(1, 20),
    'decisiontreeclassifier__max_features': ['sqrt', 'log2', None],
    'decisiontreeclassifier__criterion': ['gini', 'entropy'],
    'decisiontreeclassifier__class_weight': [None, 'balanced'],
}

search_tree = RandomizedSearchCV(
                        pipeline_tree,
                        params,
                        n_iter=50,
                        n_jobs=-1,
                        cv=5,
                        scoring='accuracy',
                        verbose=1,
                        random_state=42)

search_tree.fit(X_train,y_train)
search_tree.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


{'decisiontreeclassifier__class_weight': None,
 'decisiontreeclassifier__criterion': 'gini',
 'decisiontreeclassifier__max_depth': 7,
 'decisiontreeclassifier__max_features': None,
 'decisiontreeclassifier__min_samples_leaf': 9,
 'decisiontreeclassifier__min_samples_split': 38}

In [200]:
best_pipeline_tree = search_tree.best_estimator_

cv_results = cross_val_score(best_pipeline_tree, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7413125)

### Default Random Forest

In [201]:
# Random Forest
model_forest = RandomForestClassifier(max_depth=5)

pipeline_forest = make_pipeline(preprocessor, model_forest)
cv_results = cross_val_score(pipeline_forest, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7353124999999999)

### Optimize Random Forest

In [202]:
pipeline_forest.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num_transformer',
                                    Pipeline(steps=[('standard_scaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x1317824b0>),
                                   ('cat_transformer',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x131782210>)])),
  ('randomforestclassifier', RandomForestClassifier(max_depth=5))],
 'transform_input': None,
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('num_transformer',
                                  Pipeline(steps=[('standard_scaler',
                                                   StandardScaler())]),
                                  <sklearn.compo

In [203]:
params = {
    "randomforestclassifier__n_estimators": randint(50, 300),
    "randomforestclassifier__max_depth": randint(3, 20),
    "randomforestclassifier__min_samples_split": randint(2, 20),
    "randomforestclassifier__min_samples_leaf": randint(1, 10),
    "randomforestclassifier__max_features": ["sqrt", "log2", None],
    "randomforestclassifier__bootstrap": [True, False],
}

search_forest = RandomizedSearchCV(
                        pipeline_forest,
                        params,
                        n_iter=30,
                        n_jobs=-1,
                        cv=5,
                        scoring='accuracy',
                        verbose=1,
                        random_state=42)

search_forest.fit(X_train,y_train)
search_forest.best_params_

Fitting 5 folds for each of 30 candidates, totalling 150 fits


{'randomforestclassifier__bootstrap': True,
 'randomforestclassifier__max_depth': 13,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__min_samples_leaf': 8,
 'randomforestclassifier__min_samples_split': 5,
 'randomforestclassifier__n_estimators': 153}

In [204]:
best_pipeline_forest = search_forest.best_estimator_

cv_results = cross_val_score(best_pipeline_forest, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7440625)

### Default XGBoost

In [205]:
# XGBoost
model_xgb = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    max_depth=6,
    n_estimators=100,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

pipeline_xgb = make_pipeline(preprocessor, model_xgb)
cv_results = cross_val_score(pipeline_xgb, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7450625000000001)

### Optimize XGBoost

In [206]:
pipeline_xgb.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num_transformer',
                                    Pipeline(steps=[('standard_scaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x1317824b0>),
                                   ('cat_transformer',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x131782210>)])),
  ('xgbclassifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 feature_weights=None, gamma=None, grow_policy=None

In [207]:
params = {
    'xgbclassifier__max_depth': randint(3, 10),
    'xgbclassifier__n_estimators': randint(50, 300),
    'xgbclassifier__learning_rate': uniform(0.01, 0.3),
    'xgbclassifier__subsample': uniform(0.6, 0.4),          # 0.6～1.0
    'xgbclassifier__colsample_bytree': uniform(0.6, 0.4),    # 0.6～1.0
    'xgbclassifier__gamma': uniform(0, 5),
    'xgbclassifier__min_child_weight': randint(1, 10)
}

search_xgb = RandomizedSearchCV(
                                pipeline_xgb,
                                params,
                                n_iter=50,
                                cv=5,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=1,
                                random_state=42
                            )

search_xgb.fit(X_train, y_train)
search_xgb.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


{'xgbclassifier__colsample_bytree': np.float64(0.9560021367270265),
 'xgbclassifier__gamma': np.float64(1.689975784257679),
 'xgbclassifier__learning_rate': np.float64(0.1226748857919832),
 'xgbclassifier__max_depth': 4,
 'xgbclassifier__min_child_weight': 7,
 'xgbclassifier__n_estimators': 240,
 'xgbclassifier__subsample': np.float64(0.6143769095186968)}

In [208]:
best_pipeline_xgb = search_xgb.best_estimator_

cv_results = cross_val_score(best_pipeline_xgb, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7463749999999999)

### Default SVM

In [209]:
# SVM
model_svm = SVC(kernel='rbf', decision_function_shape='ovr', random_state=42)

pipeline_svm = make_pipeline(preprocessor, model_svm)
cv_results = cross_val_score(pipeline_svm, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7419999999999999)

### Optimize SVM

In [210]:
pipeline_svm.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num_transformer',
                                    Pipeline(steps=[('standard_scaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x1317824b0>),
                                   ('cat_transformer',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x131782210>)])),
  ('svc', SVC(random_state=42))],
 'transform_input': None,
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('num_transformer',
                                  Pipeline(steps=[('standard_scaler',
                                                   StandardScaler())]),
                                  <sklearn.compose._column_transformer.make_column

In [211]:
params = {
    "svc__C": uniform(0.1, 10),
    "svc__gamma": uniform(0.001, 0.1)
}

search_svm = RandomizedSearchCV(
    pipeline_svm,
    params,
    n_iter=20,
    n_jobs=-1,
    cv=5,
    scoring='accuracy',
    verbose=1,
    random_state=42
)

search_svm.fit(X_train, y_train)
search_svm.best_params_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/yaeyukimoto/.pyenv/versions/3.12.9/envs/project_accidents/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'svc__C': np.float64(6.942330265121568),
 'svc__gamma': np.float64(0.045015249373960134)}

In [212]:
best_pipeline_svm = search_svm.best_estimator_

cv_results = cross_val_score(best_pipeline_svm, X_train, y_train, cv=10, scoring='accuracy').mean()
cv_results

np.float64(0.7463124999999999)

## Models with optimized parameters

In [213]:
# Logistic Regression
print('Logistic Regression: ', cross_val_score(best_pipeline_lr, X_train, y_train, cv=10, scoring='accuracy').mean())

# KNN
print('KNN: ', cross_val_score(best_pipeline_knn, X_train, y_train, cv=10, scoring='accuracy').mean())

# Decision Tree
print('Decision Tree: ', cross_val_score(best_pipeline_tree, X_train, y_train, cv=10, scoring='accuracy').mean())

# Random Forest
print('Random Forest: ', cross_val_score(best_pipeline_forest, X_train, y_train, cv=10, scoring='accuracy').mean())

# XGBoost
print('XGBoost: ', cross_val_score(best_pipeline_xgb, X_train, y_train, cv=10, scoring='accuracy').mean())

# SVM
print('SVM: ', cross_val_score(best_pipeline_svm, X_train, y_train, cv=10, scoring='accuracy').mean())

Logistic Regression:  0.7404375000000001
KNN:  0.7414375
Decision Tree:  0.7413125
Random Forest:  0.7429375
XGBoost:  0.7463749999999999
SVM:  0.7463124999999999


## Final tests

In [215]:
# Logistic Regression
print('Logistic Regression: ', best_pipeline_lr.score(X_test,y_test))

# KNN
print('KNN: ', best_pipeline_knn.score(X_test,y_test))

# Decision Tree
print('Decision Tree: ', best_pipeline_tree.score(X_test,y_test))

# Random Forest
print('Random Forest: ', best_pipeline_forest.score(X_test,y_test))

# XGBoost
print('XGBoost: ', best_pipeline_xgb.score(X_test,y_test))

# SVM
print('SVM: ', best_pipeline_svm.score(X_test,y_test))

Logistic Regression:  0.73375
KNN:  0.73075
Decision Tree:  0.72275
Random Forest:  0.73225
XGBoost:  0.73375
SVM:  0.73775


---

In [ ]:
# with open('../models/project_accidents_model.pkl', 'wb') as f:
#     pickle.dump(pipe, f)

In [ ]:
# with open('../models/project_accidents_model.pkl', 'rb') as f:
#     model = pickle.load(f)